# Hamiltonian Monte Carlo (HMC)
 >__Created__:  2018 Harrison B. Prosper

The standard approach in machine learning is to search for a single optimal function within the specified function space by minimizing the average loss. In the Bayesian approach, the task is construed as a problem of *inference* in which the goal is to assign a *probability density* to every function in the function space. The higher the probability density, the better the choice of function. But, the crucial point is that one is no longer obliged to choose a single function. In principle, one can use all of them provided that results are weighted by the probability densities associated with the functions. In practice, of course, we use a finite sample of functions. 

The basic idea in Bayesian inference is to assign such probability densities (or probabilities) using Bayes theorem

$$ \boxed{\, p(w \, | \, D) = \frac{p(D \, | \, w) \, \pi(w)}{p(D)}\,}$$

The function $p(D \, | \,w)$ is called the __likelihood function__ or likelihood for short. It is the probability
model $p(x \, | \,w)$ of *potential* data $x$ evaluated at the actual data $D$. Since $D$ are constants, the likelihood is a function of the parameters $w$ only. The function $\pi(w)$ is called the prior density or __prior__ for short. It models what is known, or assumed, about $w$ independently of the data $D$. Bayes theorem reminds us of the impossibility of arriving at a satisfactory solution without making *some* assumptions. The assumptions are encoded both within the likelihood and the prior. The function $p(w \, | \, D)$ is called the __posterior density__.

Given a function $f(x, w)$, we can compute, for example, its moments

$$f_{m}(x) = \int f^m(x, w) \, p(w \, | \, D) \, dw, $$

which  can be used to assign an uncertainty to $f(x, w)$, e.g. $\sigma = \sqrt{f_2 - f_1^2}$.

### Markov Chain Sampling
In practice, integrals such as $$\int f^m(x, w) \, p(w \, | \, D) \, dw,$$ must be approximated. However, numerical integration is out of the question because the dimensionality of $w$ is too large. One approach is to approximate $p(w \, | \, D)$ by a multivariate Gaussian, but that approximation may not be good enough. The method of choice for truly hard problems is to *sample* points $w_i$ from some known probability density $g(w)$, for which sampling is fast and easy, and approximate the integral
by the average

$$f_{m}(x) \approx \frac{1}{N} \sum_{n=1}^N f^m(x, w_n) \left[\frac{p(w_n \, | \, D)}{g(w_n)} \right].$$

However, in order to achieve maximum accuracy, we should set $g(w) = p(w \, | \, D)$. But, sampling from $p(w \, | \, D)$ directly is also a hard problem! The method of choice to solve it is Hamiltonian Monte Carlo (HMC), which is a variant of Markov chain Monte Carlo (MCMC) that is well-suited to very difficult sampling problems. 

Suppose we wish to sample $w$ from the probability density $p(w)$. The quantity $w$, which could be $n$-dimensional, can be thought of as characterizing the *state* of a system. The first MCMC algorithm, developed by Metropolis, generates a sequence $w_1, w_2, \cdots$ such that in the limit of an infinite sequence, the distribution of $w_i$ converges to $p(w)$. The algorithm has two ingredients: a probabilistic rule for suggesting new states, called a __proposal function__, and the __Metropolis__ rule to decide whether or not to transition from the current state to the new. The proposal function must satisfy the principle of *detailed balance*, that is, the probability   $p(w_j \, | \, w_i) \, p(w_i)$ to transition from state $i$ to state $j$ is equal to the probability $p( w_i \, | \, w_j) \, p(w_j)$ to do the reverse. Note that if the joint probability $p(w_i, w_j)$ exists, detailed balance is an immediate consequence. 

The Metropolis rule is to accept a new state if its probability density $p(w_j)$ is greater than that of the current state $p(w_i)$, but, if the probability density of the new state is *lower*, accept the new state with probability $p(w_j) \, / \, p(w_i)$. In practice, the rule is to accept the new state if $u < p(w_j) \, / \, p(w_i)$, where $u \sim \textrm{uniform}(0, 1)$, that is, if $u$ is sampled from a uniform density on the unit interval. 

The basic idea of HMC is to map the problem of sampling from $p(w)$ to the problem of sampling the states $(q, p)$ of a particle moving in a potential $U(q)$ with kinetic energy $K(p)$, where $q$ and $p$ are the position and momentum of the particle, respectively. The states are sampled from the probability density

$$p(w) \propto g(q, p) \propto \exp(-H(q, p) \, / \, T),$$

where $H(q, p) = U(q) + K(p)$ is the __Hamiltonian__ of the system (its energy function) and $T$ the absolute temperature (which we shall set = 1). The mapping is achieved by identifying $q \equiv w$ as the position and $p \equiv (dw/dt)/m$ as the momentum and

$$U(q) = - \ln p(w).$$
 
Note that the larger the value of the Hamiltonian, the smaller the probability density $p(w)$ and, conversely, the smaller the value of $H(q, p)$ the greater the value of $p(w)$. Therefore, the Metropolis rule amounts to the following. If the particle moves to a state of lower energy, then accept that state. If it moves to a state of higher energy then accept that state with probability $p(w_j) \, / \, p(w_i)$. That is, we accept some wrong moves but with a probability that diminishes the higher the energy of the proposed new state relative to that of the current state.

In statistical mechanics, the probability density $g(q, p)$ determines the thermodynamic 
properties of the system. Of course, we should really place quotes around position, momentum, etc. because the system is fictitious!

Given a particle's Hamiltonian, its motion in classical physics can be computed using Hamilton's
equations

\begin{align*}
    \frac{dq}{dt} & = \frac{\partial H}{\partial p},\\
    \frac{dp}{dt} & = -\frac{\partial H}{\partial q},
\end{align*}

which are just Newton's laws of motion in disguise.

#### Hamiltonian Monte Carlo

Hamiltonian Monte Carlo is an MCMC algorithm with a sophisticated proposal function based on Hamilton's equations. The proposal function consists of first randomly selecting the particle's momentum then moving the particle deterministically along a finite
trajectory from state $i$ to state j in accordance with Hamiliton's equations. Then, the Metropolis rule is applied to decide whether to accept or reject the proposed state $(q, p)$. If the proposed state is rejected, we return to the start of the trajectory and try again with another randomly chosen momentum. If the state is accepted, we repeat the random selection of momentum followed by the deterministic move. The process is iterated for as many steps as needed to create a sample of points $\{(q_i, p_i)\}$ whose distribution faithfully matches $\exp(-H(q, p))$. The values $p_i$ are discarded and the $q_i$, that is, $w_i$ are kept. 

The reason HMC works is because Newton's laws are time-reversal invariant, therefore, the proposal function satisfies detailed balance. The advantage of HMC is that the degree of random walking is reduced making for a more rapid and efficient exploration of the parameter space than methods based on a pure random walk as in the original Metropolis-Hastings algorithms. Every finite trajectory is randomly oriented with respect to its predecessor, but the trajectory itself is deterministic.

The Hamilton equations are solved using a finite difference equation called the *leapfrog* algorithm. See below.


### HMC Algorithm

  1. Choose a random starting value $q = q_0$, a step size $\epsilon$, a count $L$, and a standard deviation $\sigma$.
  3. Sample $p = p_0$ from a Gaussian of zero mean and some variance $\sigma$ (p = np.random.normal(0, 1))
  3. compute
    $$p = p - \frac{\epsilon}{2} \, \frac{\partial U(q)}{\partial q}$$
  3. __for__ $i = 0\cdots L-1$ steps do:
  \begin{align*}
  q & = q + \epsilon \, p\\
    \mathbf{if} \, \, i >&= L - 1: \, \, \mathbf{break}\\
  p & =  p - \epsilon \, \frac{\partial U(q)}{\partial q}\\
     \mathbf{end \, \, for}
  \end{align*}
  5. compute
    $$p = p - \frac{\epsilon}{2} \, \frac{\partial U(q)}{\partial q}$$
  6. compute
  \begin{align*}
  U_0 & = U(q_0)\quad & K_0 = \textrm{sum}(p_0^2) \, / \, 2\\
    U & = U(q) \quad & K = \textrm{sum}(p^2) \, / \, 2\\
  \end{align*}
  7. Now decide whether to accept or reject the next point
  \begin{align*}
      u & = \textrm{np.random.uniform}(0,1)\\
      \mathbf{ if } \, \, u & < \exp(U_0 + K_0 - U - K) \, \, \mathbf{ accept } \, \, q \, \, \mathbf{ else } \, \, q = q_0
   \end{align*}
  
Steps 2 to 7 are repeated a large number of times (thousands to hundreds of thousands) and, given that it takes time for the chain to converge to the desired distribution, $p(w)$, one typically discards some fraction of the initial points and uses the rest.


In [1]:
import numpy as np

### Define a single HMC iteration
The function $U(q)$ should follow numpy semantics, namely, if $q$ is a numpy array, $U(q)$ should return a numpy array with each element the the partial derivative of $U(q)$ with respect to an element of $q$.

In [4]:
def grad(U, q, h=1.e-6):
    return 0.5*(U(q + h) - U(q - h))/h

def next_point(U, q0, epsilon=0.01, L=200, sigma=1.0):
    """
    """
    try:
        p0 = np.random.normal(np.zeros(len(q0)), sigma)
    except:
        p0 = np.random.normal(0, sigma)
    q  = q0
    p  = p0
    
    p  = p - epsilon * grad(U, q) / 2
    for i in range(L):
        q = q + epsilon * p
        if i >= L-1: break
        p = p - epsilon * grad(U, q)
    p  = p - epsilon * grad(U, q) / 2
    
    # check whether to accept or reject point
    U0 = U(q0)
    try:
        K0 = sum(p0*p0) / 2
    except:
        K0 = p0*p0 / 2
    H0 = U0 + K0
    
    U1 = U(q)
    try:
        K1  = sum(p*p) / 2
    except:
        K1  = p*p/2 
    H1 = U1 + K1
    
    u  = np.random.uniform(0,1)
    v  = np.exp(H0 - H1)
    if u < v:
        return (q, True)
    else:
        return (q0, False)